# Importaciones

In [ ]:
import random
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numpy.linalg import norm
from google.colab import files

# Definición de Variables y Funciones

In [ ]:
# Poblaciones
poblaciones = ["celtas", "iberos", "fenicios", "griegos", "italicos"]
letras = {"celtas":"C", "iberos":"I", "fenicios":"F", "griegos":"G", "italicos":"R"}

# Número de SNPs
N_snps = 48

In [ ]:
# Crear individuo a partir de probabilidades
def crear_individuo_probabilidades(probabilidades):
  pesos = [probabilidades[p] for p in poblaciones]
  snps = random.choices(poblaciones, weights=pesos, k=N_snps)
  return snps

# Reproduccir a dos individuos
def reproducir_padres(padre, madre):
    hijo = [random.choice([padre[i], madre[i]]) for i in range(N_snps)]
    return hijo

# Calcular porcentaje de cada población
def porcentaje_poblaciones(individuo):
    return {p: round(individuo.count(p)/N_snps*100, 2) for p in poblaciones}

# Mostrar SNPs como secuencia y conteo
def mostrar_snps(individuo):
    # Secuencia estilo “código de barras”
    secuencia = "".join([letras[s] for s in individuo])
    print("Secuencia SNPs:", secuencia)

    # Conteo y mini gráfico
    conteo = Counter(individuo)
    for p in poblaciones:
        print(f"{p:10}: {conteo[p]}")
    print()
    print(porcentaje_poblaciones(individuo))

In [ ]:
# --- Probabilidades---
prob_1_cantabrico = {
    'celtas': 0.95, 'iberos': 0.05, 'fenicios': 0, 'griegos': 0, 'italicos': 0}
prob_2_lusitania = {
    'celtas': 0.80, 'iberos': 0.20, 'fenicios': 0, 'griegos': 0, 'italicos': 0}
prob_3_celtiberia = {
    'celtas': 0.70, 'iberos': 0.30, 'fenicios': 0, 'griegos': 0, 'italicos': 0}
prob_4_baleares = {
    'celtas': 0.16, 'iberos': 0.75, 'fenicios': 0.04, 'griegos': 0.05, 'italicos': 0}
prob_5_turdetania = {
    'celtas': 0.12, 'iberos': 0.75, 'fenicios': 0.08, 'griegos': 0.05, 'italicos': 0}
prob_6_cartago = {
    'celtas': 0, 'iberos': 0, 'fenicios': 0.80, 'griegos': 0.10, 'italicos': 0.10}
prob_7_roma = {
    'celtas': 0.20, 'iberos': 0, 'fenicios': 0.02, 'griegos': 0.15, 'italicos': 0.63}

In [ ]:
id_num = 1
generaciones = {}
genealogia = {} # Diccionario para genealogía

# Simulación de Generaciones

## Crear Generación 1

In [ ]:
# Inicializamos la Generación 1 directamente en el diccionario principal
generaciones[1] = {"hombres": {}, "mujeres": {}}

# --- Crear los 100 individuos de la Generación 1

probabilidades_g1 = (
    [prob_3_celtiberia]*50  # 100 individuos
)

for prob in probabilidades_g1:
    # Hombre
    individuo_h = crear_individuo_probabilidades(prob)
    generaciones[1]["hombres"][f"n{id_num:03d}_g01_hom"] = individuo_h
    id_num += 1

    # Mujer
    individuo_m = crear_individuo_probabilidades(prob)
    generaciones[1]["mujeres"][f"n{id_num:03d}_g01_muj"] = individuo_m
    id_num += 1

print(f"Generación 1 creada con {len(generaciones[1]['hombres'])} hombres y {len(generaciones[1]['mujeres'])} mujeres.")

In [ ]:
# Ver algunos de los hombres
list(generaciones[1]["hombres"].keys())[:5]

In [ ]:
# Ver un individuo en concreto
mostrar_snps(generaciones[1]["mujeres"]["n052_g01_muj"])

## Crear Generación 2

In [ ]:
# Crear la estructura vacía para la generación 2
generaciones[2] = {"hombres": {}, "mujeres": {}}

# Obtener las listas de IDs de padres y madres (Generación 1)
padres_g01_ids = list(generaciones[1]["hombres"].keys())
madres_g01_ids = list(generaciones[1]["mujeres"].keys())

# Emparejar aleatoriamente (50 parejas)
parejas_g01 = list(zip(
    random.sample(padres_g01_ids, 50),
    random.sample(madres_g01_ids, 50)
))

# Cada pareja tendrá 2 hijos: 1 hombre y 1 mujer
for padre_id, madre_id in parejas_g01:
    padre_g01 = generaciones[1]["hombres"][padre_id]
    madre_g01 = generaciones[1]["mujeres"][madre_id]

    # Hijo hombre
    hijo_h_g02 = reproducir_padres(padre_g01, madre_g01)
    id_h_g02 = f"n{id_num:03d}_g02_hom"
    generaciones[2]["hombres"][id_h_g02] = hijo_h_g02
    genealogia[id_h_g02] = (padre_id, madre_id)
    id_num += 1

    # Hija mujer
    hijo_m_g02 = reproducir_padres(padre_g01, madre_g01)
    id_m_g02 = f"n{id_num:03d}_g02_muj"
    generaciones[2]["mujeres"][id_m_g02] = hijo_m_g02
    genealogia[id_m_g02] = (padre_id, madre_id)
    id_num += 1

print(f"Generación 2 creada con {len(generaciones[2]['hombres'])} hombres y {len(generaciones[2]['mujeres'])} mujeres.")

In [ ]:
print("\nMujeres G02:")
for ind_id, snps in generaciones[2]["mujeres"].items():
    print(ind_id, porcentaje_poblaciones(snps))

In [ ]:
# Ver los padres de un individuo
genealogia["n179_g02_hom"]

## Funcion Crear Generaciones

In [ ]:
def crear_generacion(
    generacion_anterior, # el diccionario de la generación previa
    num_generacion, # número entero que indica la nueva generación
    genealogia,
    id_num,
    descendencia_config,  # Lista de strings: 'hijo', 'hija', 'hijo+hija'
    inmigracion_config,   # Lista de tuplas (prob_origen, cantidad, sexo)
    num_objetivo=100
):
    nueva_generacion = {"hombres": {}, "mujeres": {}}

    # IDs de hombres y mujeres de la generación anterior
    hombres_ids = list(generacion_anterior["hombres"].keys())
    mujeres_ids = list(generacion_anterior["mujeres"].keys())
    mujeres_disponibles = mujeres_ids.copy()

# --- Emparejar automáticamente evitando hermanos ---
    parejas = []
    for padre_id in random.sample(hombres_ids, len(hombres_ids)):
        if not mujeres_disponibles:
            break
        padres_padre = genealogia[padre_id]
        mujeres_validas = [m for m in mujeres_disponibles if genealogia[m] != padres_padre]
        if not mujeres_validas:
            continue
        madre_id = random.choice(mujeres_validas)
        mujeres_disponibles.remove(madre_id)
        parejas.append((padre_id, madre_id))

    gen_tag = f"g{num_generacion:02d}"

# Configuración por defecto
    if descendencia_config is None:
        descendencia_config = (
            ['hijo+hija'] * 44 + ['hijo'] * 3 + ['hija'] * 3
        )
    random.shuffle(descendencia_config)

    gen_tag = f"g{num_generacion:02d}"

# --- Crear hijos nativos ---
    for (padre_id, madre_id), tipo in zip(parejas, descendencia_config):
        padre = generacion_anterior["hombres"][padre_id]
        madre = generacion_anterior["mujeres"][madre_id]

        if 'hijo' in tipo:
            hijo = reproducir_padres(padre, madre)
            id_h = f"n{id_num:03d}_{gen_tag}_hom"
            nueva_generacion["hombres"][id_h] = hijo
            genealogia[id_h] = (padre_id, madre_id)
            id_num += 1

        if 'hija' in tipo:
            hija = reproducir_padres(padre, madre)
            id_m = f"n{id_num:03d}_{gen_tag}_muj"
            nueva_generacion["mujeres"][id_m] = hija
            genealogia[id_m] = (padre_id, madre_id)
            id_num += 1

# --- Añadir inmigrantes ---
    total_actual = len(nueva_generacion["hombres"]) + len(nueva_generacion["mujeres"])
    faltan = num_objetivo - total_actual
    if inmigracion_config and faltan > 0:
        inmigrantes_creados = 0
        for origen, cantidad, sexo in inmigracion_config:
            for _ in range(cantidad):
                if inmigrantes_creados >= faltan:
                    break
                nuevo = crear_individuo_probabilidades(origen)
                id_new = f"n{id_num:03d}_{gen_tag}_{sexo}"
                nueva_generacion["hombres" if sexo == "hom" else "mujeres"][id_new] = nuevo
                genealogia[id_new] = ("inmigrante", list(origen.keys())[0], id_num)
                id_num += 1
                inmigrantes_creados += 1

    total_h = len(nueva_generacion["hombres"])
    total_m = len(nueva_generacion["mujeres"])
    print(f"Generación {num_generacion} creada con {total_h} hombres y {total_m} mujeres (total {total_h + total_m})")

    return nueva_generacion, genealogia, id_num

In [ ]:
config_generaciones = { # 100 individuos por generación (50 parejas)
    3: {
        "descendencia": ['hijo+hija']*40 + ['hijo']*5 + ['hija']*5, # 90 (45 hom y 45 muj)
        "inmigracion": [
            (prob_6_cartago, 3, "hom"), (prob_5_turdetania, 2, "hom"), # 5 (5 hom)
            (prob_6_cartago, 2, "muj"), (prob_1_cantabrico, 3, "muj") # 5 (5 muj)
        ]
    },
    4: {
        "descendencia": ['hijo+hija']*38 + ['hijo']*6 + ['hija']*6, # 88 (44 hom y 44 muj)
        "inmigracion": [
            (prob_7_roma, 3, "hom"), (prob_5_turdetania, 2, "hom"), (prob_1_cantabrico, 1, "hom"), # 6 (6 hom)
            (prob_1_cantabrico, 3, "muj"), (prob_4_baleares, 2, "muj"), (prob_5_turdetania, 1, "muj") # 6 (6 muj)
        ]
    },
    5: {
        "descendencia": ['hijo+hija']*38 + ['hijo']*6 + ['hija']*6, # 88 (44 hom y 44 muj)
        "inmigracion": [
            (prob_1_cantabrico, 3, "hom"), (prob_7_roma, 3, "hom"), # 6 (6 hom)
            (prob_1_cantabrico, 3, "muj"), (prob_2_lusitania, 3, "muj") # 6 (6 muj)
        ]
    },
    6: {
        "descendencia": ['hijo+hija']*38 + ['hijo']*6 + ['hija']*6, # 88 (44 hom y 44 muj)
        "inmigracion": [
            (prob_7_roma, 3, "hom"), (prob_1_cantabrico, 3, "hom"), # 6 (6 hom)
            (prob_7_roma, 3, "muj"), (prob_2_lusitania, 3, "muj") # 6 (6 muj)
        ]
    },
    7: {
        "descendencia": ['hijo+hija']*38 + ['hijo']*6 + ['hija']*6, # 88 (44 hom y 44 muj)
        "inmigracion": [
            (prob_1_cantabrico, 3, "hom"), (prob_2_lusitania, 3, "hom"), # 6 (6 hom)
            (prob_2_lusitania, 3, "muj"), (prob_7_roma, 3, "muj") # 6 (6 muj)
        ]
    },
    8: {
        "descendencia": ['hijo+hija']*36 + ['hijo']*7 + ['hija']*7, # 86 (43 hom y 43 muj)
        "inmigracion": [
            (prob_1_cantabrico, 5, "hom"), (prob_7_roma, 2, "hom"), # 7 (7 hom)
            (prob_1_cantabrico, 5, "muj"), (prob_2_lusitania, 2, "muj") # 7 (7 muj)
        ]
    },
    9: {
        "descendencia": ['hijo+hija']*40 + ['hijo']*5 + ['hija']*5, # 90 (45 hom y 45 muj)
        "inmigracion": [
            (prob_1_cantabrico, 3, "hom"), (prob_4_baleares, 1, "hom"), (prob_6_cartago, 1, "hom"), # 5 (5 hom)
            (prob_1_cantabrico, 1, "muj"), (prob_2_lusitania, 2, "muj"), (prob_7_roma, 1, "muj"), (prob_5_turdetania, 1, "muj") # 5 (5 muj)
        ]
    }
}

def validar_config(cfg):
    print("Validando configuración...\n")
    for gen, conf in cfg.items():
        desc = conf["descendencia"]
        inm = conf["inmigracion"]
        n_parejas = len(desc)

        # contar hijos
        n_hijos = sum(2 if d == "hijo+hija" else 1 for d in desc)
        n_inm = sum(n for _, n, _ in inm)
        total = n_hijos + n_inm
        print(f"Gen {gen}: parejas={n_parejas}, hijos={n_hijos}, inmigrantes={n_inm}, total={total}")
    print("\nRevisión completa.")

# ejecutar:
validar_config(config_generaciones)


In [ ]:
# Generar G3–G8 automáticamente
for g in range(3, 10):  # 3 a 9 inclusive
    config = config_generaciones[g]  # siempre existe
    generaciones[g], genealogia, id_num = crear_generacion(
        generacion_anterior=generaciones[g-1],
        num_generacion=g,
        genealogia=genealogia,
        id_num=id_num,
        descendencia_config=config["descendencia"],
        inmigracion_config=config["inmigracion"],
        num_objetivo=100
    )

# Visualizar

In [ ]:
# Funciones para crear Dataframes
def generar_df_generacion_completa(generacion):
    data = []
    for sexo in ["hombres", "mujeres"]:
        for id_ind, snps in generacion[sexo].items():
            pcts = porcentaje_poblaciones(snps)  # usamos tu función
            pcts["ID"] = id_ind
            data.append(pcts)
    df = pd.DataFrame(data)
    return df

In [ ]:
# Creamos diccionario llamado dfs_generaciones, que contendrá un DataFrame por cada generación
dfs_generaciones = {}
for gen_num, gen_data in generaciones.items():
    dfs_generaciones[gen_num] = generar_df_generacion_completa(gen_data)

In [ ]:
dfs_generaciones[9]

In [ ]:
# Creo un diccionario para guardar los promedios de cada origen por generación
medias_generaciones = {p: [] for p in poblaciones}

for gen_num in sorted(dfs_generaciones.keys()):
    df = dfs_generaciones[gen_num]
    for p in poblaciones:
        medias_generaciones[p].append(df[p].mean())

In [ ]:
# Graficar la evolución de cada origen
plt.figure(figsize=(10,6))

for p in poblaciones:
    plt.plot(
        sorted(dfs_generaciones.keys()),     # Eje X → número de generación
        medias_generaciones[p],              # Eje Y → promedio de ese origen
        marker='o',
        label=p.capitalize()
    )

plt.title("Evolución de la composición genética media por generación")
plt.xlabel("Generación")
plt.ylabel("Porcentaje medio (%)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# --- Calcular medias y varianzas de todas las generaciones ---
df_media = pd.DataFrame(medias_generaciones).T
df_media.columns = sorted(dfs_generaciones.keys())
df_varianza = pd.DataFrame({gen: df.drop(columns=['ID']).var() for gen, df in dfs_generaciones.items()})

numero_1_comparacion = 3
numero_2_comparacion = 9

# --- Seleccionar solo esas generaciones ---
df_media_comparacion_generaciones = df_media[[numero_1_comparacion, numero_2_comparacion]]
df_varianza_comparacion_generaciones = df_varianza[[numero_1_comparacion, numero_2_comparacion]]

# --- Graficar ---
x = np.arange(len(df_media_comparacion_generaciones))  # posiciones en eje x (una por población)
ancho = 0.3  # ancho de cada barra

plt.figure(figsize=(10,6))

# Barras con desviación estándar como error bars
plt.bar(x - ancho/2, df_media_comparacion_generaciones[numero_1_comparacion], width=ancho,
        yerr=np.sqrt(df_varianza_comparacion_generaciones[numero_1_comparacion]), capsize=5, label=f'Generación {numero_1_comparacion}', color='orange')

plt.bar(x + ancho/2, df_media_comparacion_generaciones[numero_2_comparacion], width=ancho,
        yerr=np.sqrt(df_varianza_comparacion_generaciones[numero_2_comparacion]), capsize=5, label=f'Generación {numero_2_comparacion}', color='green')

# Configuración de ejes y título
plt.xticks(x, df_media_comparacion_generaciones.index, rotation=45)
plt.ylabel("Porcentaje promedio (%)")
plt.title(f"Comparación Generación {numero_1_comparacion} vs Generación {numero_2_comparacion}\n(Media ± Desviación estándar)")
plt.ylim(0, 100)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Muestra los porcentajes de dos generaciones en un heatmap
numero_1_comparacion = 3
numero_2_comparacion = 9

# --- Extraer medias de df_media ---
df_heat = df_media[[numero_1_comparacion, numero_2_comparacion]].copy()
df_heat.columns = [f"G{numero_1_comparacion}", f"G{numero_2_comparacion}"]

# --- Crear heatmap ---
plt.figure(figsize=(8,6))
sns.heatmap(df_heat, annot=True, cmap="YlOrRd", fmt=".2f", linewidths=.5)
plt.title(f"Comparación porcentual de orígenes entre Generación {numero_1_comparacion} y {numero_2_comparacion}", fontsize=14)
plt.ylabel("Origen")
plt.xlabel("Generación")
plt.show()

In [ ]:
# Muestra todas las generaciones en una sola gráfica
df_heat = df_media.copy()
df_heat.columns = [f"G{c}" for c in df_heat.columns]

# --- Crear heatmap ---
plt.figure(figsize=(10,6))
sns.heatmap(df_heat, annot=True, cmap="YlOrRd", fmt=".2f", linewidths=.5)
plt.title("Evolución de los porcentajes medios de cada origen por generación", fontsize=14)
plt.ylabel("Origen poblacional")
plt.xlabel("Generación")
plt.show()

In [ ]:
# --- Calcular el cambio porcentual entre generaciones consecutivas ---
df_cambio = df_heat.diff(axis=1)  # resta columnas consecutivas: G2-G1, G3-G2, etc.
df_cambio = df_cambio.iloc[:, 1:]  # quitamos la primera (G1 no tiene anterior)

# --- Crear heatmap de diferencias ---
plt.figure(figsize=(10,6))
sns.heatmap(df_cambio, annot=True, cmap="coolwarm", center=0, fmt=".2f", linewidths=.5)
plt.title("Cambio porcentual de los orígenes entre generaciones consecutivas", fontsize=14)
plt.ylabel("Origen poblacional")
plt.xlabel("Generaciones (comparadas con la anterior)")
plt.show()

# Distancia ancestral

In [ ]:
# Selecciona la generación 1 y quita columna ID
df_gen1 = dfs_generaciones[1].drop(columns=["ID"])

# Vector medio de la generación 1
vector_medio_g1 = df_gen1.mean()

print("Vector medio de la generación 1:")
print(vector_medio_g1)

In [ ]:
# concatenar todas las generaciones
df_all_gen = pd.concat(
    [df.assign(generacion=gen) for gen, df in dfs_generaciones.items()],
    ignore_index=True
)

# calcular distancia euclídea respecto al vector medio de G1
df_all_gen["distancia_G1"] = df_all_gen[poblaciones].apply(
    lambda fila: norm(fila - vector_medio_g1), axis=1
)

In [ ]:
# Filtrar generaciones de la 2 a la 8
df_gen_2_8 = df_all_gen[df_all_gen["generacion"].between(2, 8)].drop(columns=["generacion"]).copy()

# Ordenar por distancia creciente
df_ordenado = df_gen_2_8.sort_values(by="distancia_G1")

# 5 individuos más cercanos a G1
mas_cercanos = df_ordenado.head(5)
print("----- 5 individuos más cercanos a G1 -----")
print(mas_cercanos)

# 5 individuos más lejanos a G1
mas_lejanos = df_ordenado.tail(5)
print("\n----- 5 individuos más lejanos a G1 -----")
print(mas_lejanos)

In [ ]:
# Filtrar solo generacion 9
df_gen9 = df_all_gen[df_all_gen["generacion"] == 9].drop(columns=["generacion"]).copy()

# Ordenar por distancia creciente
df_ordenado = df_gen9.sort_values(by="distancia_G1")

# 5 individuos más cercanos a G1
mas_cercanos = df_ordenado.head(5)
print("----- 5 individuos más cercanos a G1 -----")
print(mas_cercanos)

# 5 individuos más lejanos a G1
mas_lejanos = df_ordenado.tail(5)
print("\n----- 5 individuos más lejanos a G1 -----")
print(mas_lejanos)

In [ ]:
n_total = len(df_gen9)

df_gen9["pct_peninsular"] = df_gen9["celtas"] + df_gen9["iberos"]
cond_totalmente_peninsular = df_gen9["pct_peninsular"] >= 99.9
cond_mayorit_peninsular = (df_gen9["pct_peninsular"] >= 80) & (df_gen9["pct_peninsular"] <99.9)
cond_mix_peninsular = (df_gen9["pct_peninsular"] >=0.1) & (df_gen9["pct_peninsular"] <80)
cond_no_peninsulares = df_gen9["pct_peninsular"] <0.1

In [ ]:
# Contar individuos en cada grupo
n_totalmente = cond_totalmente_peninsular.sum()
n_mayorit = cond_mayorit_peninsular.sum()
n_mix = cond_mix_peninsular.sum()
n_no = cond_no_peninsulares.sum()

# Calcular porcentajes
pct_totalmente = n_totalmente / n_total * 100
pct_mayorit = n_mayorit / n_total * 100
pct_mix = n_mix / n_total * 100
pct_no = n_no / n_total * 100

In [ ]:
# Preparar datos para el gráfico
labels = [
    "Peninsulares (>99.9%)",
    "Mayormente peninsulares (80% – 99.9%)",
    "Mixtos (0.1% – 79.9%)",
    "No peninsulares (<0.1%)"
]
sizes = [pct_totalmente, pct_mayorit, pct_mix, pct_no]
colors = ["#27AE60", "#F4D03F", "#5DADE2", "#E74C3C"]

# Gráfico de pastel
fig, ax = plt.subplots(figsize=(6, 6))
wedges, texts, autotexts = ax.pie(
    sizes,
    labels=None,  # no etiquetas dentro
    autopct=lambda p: f"{p:.1f}%",
    startangle=90,
    colors=colors,
    pctdistance=1.1,  # pone porcentajes fuera
    wedgeprops={"edgecolor": "white"}
)

ax.axis("equal")  # mantener círculo

# Leyenda
ax.legend(wedges, labels, loc="center left", bbox_to_anchor=(1, 0.5), frameon=False)

# Título
ax.set_title("Agrupación de Individuos según su % de genética peninsular - Generación 9", fontsize=14, pad=15)

plt.show()

In [ ]:
# Guardar el DataFrame en un CSV
df_all_gen.to_csv("todas_las_generaciones.csv", index=False, encoding="utf-8")
# Descargar el archivo
files.download("todas_las_generaciones.csv")

In [ ]:
df_gen_2_8.to_csv("generaciones_2_a_8.csv", index=False, encoding="utf-8")
files.download("generaciones_2_a_8.csv")

In [ ]:
df_gen9_des = df_gen9.drop(columns=["pct_peninsular"]).copy()

In [ ]:
df_gen9_des.to_csv("generacion_9.csv", index=False, encoding="utf-8")
files.download("generacion_9.csv")

In [ ]:
df_media.to_csv("medias_generaciones.csv", index=False, encoding="utf-8")
files.download("medias_generaciones.csv")

In [ ]:
df_varianza.to_csv("varianza_generaciones.csv", index=False, encoding="utf-8")
files.download("varianza_generaciones.csv")

In [ ]:
df_cambio.to_csv("cambio_generaciones.csv", index=False, encoding="utf-8")
files.download("cambio_generaciones.csv")